In [ ]:
!pip3 install chromadb

In [1]:
import chromadb

import textwrap
import os

In [ ]:
client = chromadb.Client()

In [ ]:
collection = client.create_collection(name="demo")

In [4]:
data_dir = "./stackoverflow_data/"
if not os.path.isdir(data_dir):
    os.system("mkdir "+data_dir)

os.system("wget -nv -O "+data_dir+"train.csv 'https://www.dropbox.com/scl/fi/e7ltns3teqmngai3fp71o/stackoverflow_train.csv?rlkey=f9pq0amo5swfrdi1hcvk52cuc&dl=0'")
os.system("wget -nv -O "+data_dir+"test.csv 'https://www.dropbox.com/scl/fi/9rqf0nf2cmti0ihn5x4uu/stackoverflow_test.csv?rlkey=cn3ttfenhgxrsaemidnat0kt2&dl=0'")

2024-03-11 10:01:18 URL:https://ucd55e10d6121fa273f6098da9a2.dl.dropboxusercontent.com/cd/0/inline/CO7-1xMQb8A09-lu2YuWwssOx8IigZIT68-sswhKdrhnQPYQgOOKpy81tV-wfF3hF30W-RZejLJpQO7P80FS5YM5SvIrGxt7h8OptTCjpWnfGyEZwYRNkemUjqesmH_flRaw1gmaytdi8d2HaNtjzz6v/file [26625547/26625547] -> "./stackoverflow_data/train.csv" [1]
2024-03-11 10:01:19 URL:https://uc53e2d8f0465990ef478106ecf7.dl.dropboxusercontent.com/cd/0/inline/CO4fDAaq7oqxmvStXX0dmzAnoApzpkgfnZ3gXEXF41AHFB3fn-E99enpgvp7afte3rhaInlgP6WIPN8D_MCUxPJ3GXdh6Q6Qu5j-PZeSWZ-4pdi8IqHXVKfE5Mo4wu4OAsBMH13kuM1TZ3y-YM49iIh4/file [136081/136081] -> "./stackoverflow_data/test.csv" [1]


0

In [5]:
train_file = data_dir+"train.csv"
test_file = data_dir+"test.csv"

In [6]:
import pandas as pd
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [7]:
train_answers = train_df['answer'].to_list()
answer_ids = train_df['id'].to_list()

In [ ]:
from tqdm import tqdm
total_docs = len(train_answers)
max_batch_size = 512  
for start_idx in tqdm(range(0, total_docs, max_batch_size)):
    end_idx = min(start_idx + max_batch_size, total_docs)
    batch_documents = train_answers[start_idx:end_idx]
    batch_ids = answer_ids[start_idx:end_idx]
    collection.add(documents=batch_documents, ids=list(map(str, batch_ids)))

In [9]:
def precision_at_k(predicted, original, k):
    total_precision = 0
    for pred_instance, orig_instance in zip(predicted, original):
        total_precision += len(set(pred_instance[:k]) & set(orig_instance)) / k
    return total_precision / len(predicted) 
questions = test_df['query'].to_list()
true_labels_all = list(map(lambda x: list(map(int, x.split(","))), test_df['ids'].to_list()))
predicted_all, original_all, original_top  = [], [], []
results = collection.query(query_texts=questions,n_results=5)['ids']
for result, true_labels in list(zip(results, true_labels_all)):
    predicted_all.append(list(map(int, result)))
    original_all.append(true_labels)
    original_top.append(true_labels[:1])
print("Relevance Precision@1 =", precision_at_k(predicted_all, original_all, 1)) 
print("Helpfulness Precision@1 =", precision_at_k(predicted_all, original_top, 1)) 

Failed to send telemetry event CollectionQueryEvent: module 'chromadb' has no attribute 'get_settings'


Relevance Precision@1 = 0.45226130653266333
Helpfulness Precision@1 = 0.37185929648241206


In [10]:
wrapper = textwrap.TextWrapper(width=100)

In [11]:
query = """I possess an algorithm that produces strings from a list of input words. 
How do I isolate only the strings that sound like English words? ie. reject 
RDLO while retaining LORD. EDIT: To clarify, they don't have to be real words 
in the dictionary. They just need to resemble English. For instance, KEAL would be 
acceptable."
"""


results = collection.query(query_texts=[query],n_results=1)
for answer in results['documents'][0]:
    wrapped_text = wrapper.wrap(text = answer)
    for element in wrapped_text:
        print(element)
    print()

 Since you are searching for whole words, how about searching for \w+ , and checking if the word is
in a collection. A hash-based set or a hash-map would work well here. This approach would make it
easier to update the list if the need should arise.



In [12]:
query = """What is the optimal way to unit test a method that
invokes multiple methods, for instance: modify(string value) 
{ if(value.Length &gt; 5) replaceit(value); else 
changeit(value); } This pseudo code has a modify method 
that (currently) calls either replaceit() or changeit() . 
I have already written tests for replaceit and changeit , 
so creating a new test for modify will be 99% the same set 
of code. I need to test it though because it might change 
in the future. So do I duplicate the existing test code? 
Shift the test code to a common function? Any other 
suggestions? I'm uncertain of the best practice here.
"""

results = collection.query(query_texts=[query],n_results=1)
for answer in results['documents'][0]:
    wrapped_text = wrapper.wrap(text = answer)
    for element in wrapped_text:
        print(element)
    print()

 Tests that check for more than one thing aren't usually recommended because they are more tightly
coupled and brittle. If you change something in the code, it'll take longer to change the test,
since there are more things to account for. [Edit:] Ok, say this is a sample test method:
[TestMethod] public void TestSomething() { // Test condition A // Test condition B // Test condition
C // Test condition D } If your test for condition A fails, then B, C, and D will appear to fail as
well, and won't provide you with any usefulness. What if your code change would have caused C to
fail as well? If you had split them out into 4 separate tests, you would know this.



In [13]:
query = """I am seeking to develop a very lightweight GUI front end in Windows.
It's meant to perform a simple task - when a hot key combination is pressed it
opens up a text box. Any text can be pasted in and then saved with a simple text 
box. I am aiming to avoid any menu bar or toolbars completely. What would be the
perfect GUI library to create something like this?"""

results = collection.query(query_texts=[query],n_results=2)
for answer in results['documents'][0]:
    wrapped_text = wrapper.wrap(text = answer)
    for element in wrapped_text:
        print(element)
    print()

 Have a look at Qt . It is an open source library for making GUI's. Unlike Swing in Java, it assumes
a lot of stuff, so it is really easy to make functional GUI's. For example, a textarea assumes that
you want a context menu when you right click it with copy, paste, select all, etc. The documentation
is also very good.

 Visual Editor for quick GUI development.

